In [1]:
import csv
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import DCTERMS, RDF, RDFS, SKOS, XSD
import pandas as pd
import numpy

In [2]:
movies = pd.read_csv("movies_metadata.csv")
movies.head(1)

/tmp/ipykernel_86495/257150559.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [3]:
mov = movies.loc[:,["id", "genres", "original_language", "production_countries", "title","release_date"]]
mov = mov.dropna()
mov

,id,genres,original_language,production_countries,title,release_date
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Toy Story,1995-10-30
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Jumanji,1995-12-15
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Grumpier Old Men,1995-12-22
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Waiting to Exhale,1995-12-22
4,11862,"[{'id': 35, 'name': 'Comedy'}]",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Father of the Bride Part II,1995-02-10
...,...,...,...,...,...,...
45460,30840,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",en,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",Robin Hood,1991-05-13
45462,111109,"[{'id': 18, 'name': 'Drama'}]",tl,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",Century of Birthing,2011-11-17
45463,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Betrayal,2003-08-01
45464,227506,[],en,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",Satan Triumphant,1917-10-21


In [4]:
mov['release_date'] = pd.to_datetime(mov['release_date'])

# Sélectionner uniquement l'année
mov['year'] = mov['release_date'].dt.year
mov = mov.drop('release_date', axis=1)

mov

,id,genres,original_language,production_countries,title,year
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Toy Story,1995
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Jumanji,1995
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Grumpier Old Men,1995
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Waiting to Exhale,1995
4,11862,"[{'id': 35, 'name': 'Comedy'}]",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Father of the Bride Part II,1995
...,...,...,...,...,...,...
45460,30840,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",en,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",Robin Hood,1991
45462,111109,"[{'id': 18, 'name': 'Drama'}]",tl,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",Century of Birthing,2011
45463,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,"[{'iso_3166_1': 'US', 'name': 'United States o...",Betrayal,2003
45464,227506,[],en,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",Satan Triumphant,1917


In [5]:
import langcodes

# Liste des codes de langue
lang_codes = mov['original_language']

l = []

# Boucle pour obtenir le nom complet de chaque langue
for i in lang_codes:
    lang_name = langcodes.Language.get(i).language_name()
    l.append(lang_name)

mov['language'] = l
mov = mov.drop('original_language', axis=1)

mov

,id,genres,production_countries,title,year,language
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",Toy Story,1995,English
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",Jumanji,1995,English
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'iso_3166_1': 'US', 'name': 'United States o...",Grumpier Old Men,1995,English
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",Waiting to Exhale,1995,English
4,11862,"[{'id': 35, 'name': 'Comedy'}]","[{'iso_3166_1': 'US', 'name': 'United States o...",Father of the Bride Part II,1995,English
...,...,...,...,...,...,...
45460,30840,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",Robin Hood,1991,English
45462,111109,"[{'id': 18, 'name': 'Drama'}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",Century of Birthing,2011,Filipino
45463,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",Betrayal,2003,English
45464,227506,[],"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",Satan Triumphant,1917,English


In [6]:
import ast
import json

serie_liste = mov['genres']
genres = []

# Convertir la chaîne en une liste Python
ma_liste = [ast.literal_eval(element) for element in serie_liste]

# Ne garder que les genres
for item in ma_liste:
    
    lst = []
    for el in item:
        objet = json.loads(str(el).replace("'", "\""))
        nom = objet["name"]
        lst.append(nom)
    genres.append(lst)
        
genres

[['Animation', 'Comedy', 'Family'],
 ['Adventure', 'Fantasy', 'Family'],
 ['Romance', 'Comedy'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy'],
 ['Action', 'Crime', 'Drama', 'Thriller'],
 ['Comedy', 'Romance'],
 ['Action', 'Adventure', 'Drama', 'Family'],
 ['Action', 'Adventure', 'Thriller'],
 ['Adventure', 'Action', 'Thriller'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Horror'],
 ['Family', 'Animation', 'Adventure'],
 ['History', 'Drama'],
 ['Action', 'Adventure'],
 ['Drama', 'Crime'],
 ['Drama', 'Romance'],
 ['Crime', 'Comedy'],
 ['Crime', 'Comedy', 'Adventure'],
 ['Action', 'Comedy', 'Crime'],
 ['Comedy', 'Thriller', 'Crime'],
 ['Drama', 'Thriller'],
 ['Action', 'Adventure', 'Crime', 'Thriller'],
 ['Drama', 'Fantasy', 'Science Fiction', 'Thriller'],
 ['Drama', 'Romance'],
 ['Drama'],
 ['Comedy', 'Drama', 'Family'],
 ['Drama', 'Romance'],
 ['Fantasy', 'Science Fiction', 'Adventure'],
 ['Drama', 'Crime'],
 ['Drama', 'Crime'],
 ['Science Fiction', 'Thriller', 'Mystery'],
 ['Romance

In [7]:
exists = set(["Thriller", "Crime", "Action", "Drama", "Comedy"])
authorized = []

for item in genres:
    
    lst = []
    for el in item:
        if el in exists:
            lst.append(el)
    authorized.append(lst)
    
authorized

[['Comedy'],
 [],
 ['Comedy'],
 ['Comedy', 'Drama'],
 ['Comedy'],
 ['Action', 'Crime', 'Drama', 'Thriller'],
 ['Comedy'],
 ['Action', 'Drama'],
 ['Action', 'Thriller'],
 ['Action', 'Thriller'],
 ['Comedy', 'Drama'],
 ['Comedy'],
 [],
 ['Drama'],
 ['Action'],
 ['Drama', 'Crime'],
 ['Drama'],
 ['Crime', 'Comedy'],
 ['Crime', 'Comedy'],
 ['Action', 'Comedy', 'Crime'],
 ['Comedy', 'Thriller', 'Crime'],
 ['Drama', 'Thriller'],
 ['Action', 'Crime', 'Thriller'],
 ['Drama', 'Thriller'],
 ['Drama'],
 ['Drama'],
 ['Comedy', 'Drama'],
 ['Drama'],
 [],
 ['Drama', 'Crime'],
 ['Drama', 'Crime'],
 ['Thriller'],
 [],
 ['Drama', 'Comedy'],
 ['Drama'],
 ['Drama'],
 ['Drama'],
 ['Comedy'],
 ['Comedy', 'Drama'],
 ['Drama'],
 ['Drama'],
 ['Action', 'Crime', 'Drama'],
 ['Drama'],
 ['Action'],
 ['Drama', 'Comedy', 'Thriller'],
 ['Drama'],
 ['Crime', 'Thriller'],
 ['Drama'],
 ['Drama'],
 ['Drama', 'Crime', 'Thriller'],
 ['Action', 'Thriller', 'Drama'],
 ['Comedy'],
 ['Drama'],
 ['Action', 'Comedy'],
 ['Drama'

In [8]:
mov['genres'] = authorized
mov = mov.drop(mov[mov['genres'].apply(len) == 0].index)
mov

,id,genres,production_countries,title,year,language
0,862,[Comedy],"[{'iso_3166_1': 'US', 'name': 'United States o...",Toy Story,1995,English
2,15602,[Comedy],"[{'iso_3166_1': 'US', 'name': 'United States o...",Grumpier Old Men,1995,English
3,31357,"[Comedy, Drama]","[{'iso_3166_1': 'US', 'name': 'United States o...",Waiting to Exhale,1995,English
4,11862,[Comedy],"[{'iso_3166_1': 'US', 'name': 'United States o...",Father of the Bride Part II,1995,English
5,949,"[Action, Crime, Drama, Thriller]","[{'iso_3166_1': 'US', 'name': 'United States o...",Heat,1995,English
...,...,...,...,...,...,...
45454,420346,"[Comedy, Drama]","[{'iso_3166_1': 'US', 'name': 'United States o...",The Morning After,2015,English
45456,84419,[Thriller],"[{'iso_3166_1': 'US', 'name': 'United States o...",House of Horrors,1946,English
45460,30840,"[Drama, Action]","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",Robin Hood,1991,English
45462,111109,[Drama],"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",Century of Birthing,2011,Filipino


In [9]:
import ast

mov['production_countries'] = mov['production_countries'].apply(lambda x: ast.literal_eval(x))
mov['production_countries'] = mov['production_countries'].apply(lambda x: x[0]['name'] if len(x) > 0 else None)

mov = mov.dropna(subset=['production_countries'])

mov = mov.drop_duplicates(subset='id')
mov = mov.drop_duplicates(subset='title')
mov = mov.rename(columns={'production_countries': 'country'})
mov

,id,genres,country,title,year,language
0,862,[Comedy],United States of America,Toy Story,1995,English
2,15602,[Comedy],United States of America,Grumpier Old Men,1995,English
3,31357,"[Comedy, Drama]",United States of America,Waiting to Exhale,1995,English
4,11862,[Comedy],United States of America,Father of the Bride Part II,1995,English
5,949,"[Action, Crime, Drama, Thriller]",United States of America,Heat,1995,English
...,...,...,...,...,...,...
45446,49271,[Comedy],France,The Devilish Tenant,1909,English
45450,49280,"[Action, Thriller]",France,The One-Man Band,1900,French
45451,106807,[Comedy],France,The Fat and Lean Wrestling Match,1900,French
45456,84419,[Thriller],United States of America,House of Horrors,1946,English


In [10]:
mov.to_csv('mov.csv', index=False)

In [19]:
import ast
from owlready2 import *
import pandas as pd

# Load the ontology
onto = get_ontology("Ontology1.owl").load()

# Define the classes in the ontology
print(list(onto.classes()))
df = pd.read_csv('mov.csv')
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x))
# Loop through the rows of the CSV file and create instances of the Movie class
for index, row in df.iterrows():

    ##print(row)
    # Create instances of the Genre class
    #print("Number of instances before:", len(list(onto.individuals())))
    #print("1")
    genres = []
    for g in row['genres']:
        match g:
            case 'Action':
                print("hi")
                genres.append(onto.Action)
            case 'Drama':
                print("hi")
                genres.append(onto.Drama)
            case 'Comedy':
                print("hi")
                genres.append(onto.Comedy)
            case 'Thriller':
                print("hi")
                genres.append(onto.Thriller)
            case 'Crime':
                print("hi")
                genres.append(onto.Crime)
            case _:
                print(index)
                pass
    #print("2")
    # Create instances of the Person class
    #directors = [Person(Name=d.strip(), Gender='Male') for d in row['directors'].strip("[]").split(",")]
    #writers = [Person(Name=w.strip(), Gender='Male') for w in row['writers'].strip("[]").split(",")]
    #actors = [Person(Name=a.strip(), Gender='Male') for a in row['actors'].strip("[]").split(",")]
    print(genres)
    #print("3")
    # Create an instance of the Movie class and add the directors, writers, actors, genres, and other properties
    print(row['title'].strip())
    movie = onto.Movie(Title=row['title'].strip())  
    #print("4")
    #movie.hasDirector = directors
    #movie.hasWriter = writers
    #movie.hasActor = actors
    movie.hasGenre = genres
    movie.Year = [row['year']]
    movie.Country = [row['country']]
    movie.Language = [row['language']]
    #print("5")
    # Add the instance to the ontology
    print("hey")

    onto.save(file="Imported1.owl", format = "rdfxml")
    #print("Number of instances after:", len(list(onto.individuals())))



[untitled-ontology-21.Action, untitled-ontology-21.Actor, untitled-ontology-21.Comedy, untitled-ontology-21.Crime, untitled-ontology-21.Director, untitled-ontology-21.Drama, untitled-ontology-21.Movie, untitled-ontology-21.Person, untitled-ontology-21.Thriller, untitled-ontology-21.Writer, untitled-ontology-48.Genre, untitled-ontology-48.Theater]
hi


TypeError: 'NoneType' object is not callable